In [ ]:
!pip install objectnat iduedu

In [ ]:
%load_ext autoreload
%autoreload 2
from iduedu import get_intermodal_graph,get_boundary
poly = get_boundary(osm_id=1114252)
G_intermodal = get_intermodal_graph(polygon=poly.buffer(0.02))
# Retrieving the intermodal graph for a specific region using OSM ID 1114252. This graph includes transportation networks
# (such as roads, public transport routes) for a given area.

In [ ]:

from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrones

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data
# Point(30.295606, 59.9439234),Point(30.27060176,59.93546846)

points = gpd.GeoDataFrame(geometry=[Point(30.27060176,59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.295606 and latitude 59.9439234).


isochrones_r, stops, routes = get_accessibility_isochrones(
    isochrone_type='radius',
    points=points,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal
)


# Visualization
m = isochrones_r.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')

In [ ]:
isochrones_w, stops, routes = get_accessibility_isochrones(
    isochrone_type='ways',
    points=points,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal
)


# Visualization
m = isochrones_w.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')

In [ ]:
m = isochrones_r.explore(tiles='CartoDB positron')
isochrones_w.explore(m=m,color='red')

In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data
# Point(30.295606, 59.9439234),Point(30.27060176,59.93546846)

point = gpd.GeoDataFrame(geometry=[Point(30.27060176,59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='separate',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=3
)
result.explore(tiles='CartoDB positron',column='dist')


In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data
# Point(30.295606, 59.9439234),Point(30.27060176,59.93546846)

point = gpd.GeoDataFrame(geometry=[Point(30.27060176,59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='radius',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=0.5
)
result.explore(tiles='CartoDB positron',column='dist')


In [95]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data
# Point(30.295606, 59.9439234),Point(30.27060176,59.93546846)

point = gpd.GeoDataFrame(geometry=[Point(30.27060176,59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='ways',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=3
)
result.explore(tiles='CartoDB positron',column='dist')

2025-04-07 18:13:50.217 | INFO     | objectnat.methods.isochrones.isochrones:get_accessibility_isochrone_stepped:60 - Building isochrones geometry...


[ 0  3  6  9 10]


In [93]:
result

,x,y,desc,route,geometry,dist
32399,346335.412936,6.645846e+06,NaN,NaN,POINT (346335.413 6645845.926),8.588111
38409,346289.670273,6.645868e+06,NaN,NaN,POINT (346289.67 6645868.408),9.200111
32400,346261.717800,6.645879e+06,NaN,NaN,POINT (346261.718 6645878.936),9.560111
39585,346388.230864,6.645882e+06,NaN,NaN,POINT (346388.231 6645881.785),7.911111
32398,346228.043711,6.645915e+06,NaN,NaN,POINT (346228.044 6645914.877),9.875111
...,...,...,...,...,...,...
11244,348777.537625,6.649121e+06,NaN,NaN,POINT (348777.538 6649120.623),9.716111
7945,348772.639219,6.649123e+06,NaN,NaN,POINT (348772.639 6649123.499),9.784111
38447,348634.255530,6.649346e+06,NaN,NaN,POINT (348634.256 6649346.476),9.575111
492,348643.469759,6.649349e+06,platform,"1,191,227",POINT (348643.47 6649349.486),9.409111


In [91]:
result = get_accessibility_isochrone_stepped(
    isochrone_type='ways',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=1
)
result.explore(tiles='CartoDB positron',column='dist')

2025-04-07 18:07:45.708 | INFO     | objectnat.methods.isochrones.isochrones:get_accessibility_isochrone_stepped:60 - Building isochrones geometry...


[ 0  1  2  3  4  5  6  7  8  9 10]
